# 기사 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from newspaper import Article
from konlpy.tag import Okt
import pandas as pd
import time
from tqdm import tqdm
import nltk

# Download NLTK stopwords
nltk.download('stopwords')

# Create an Okt object
okt = Okt()

# Selenium options
options = Options()
prefs = {
    "download.default_directory": "/home/dev_ws/eda",
    "download.prompt_for_download": False
}
options.add_experimental_option('prefs', prefs)

# Path to the ChromeDriver
webdriver_service = Service("../eda/driver/chromedriver")

# Start the WebDriver
driver = webdriver.Chrome(service=webdriver_service, options=options)

# Naver news search URL
search_url = """https://search.naver.com/search.naver?where=news&query=%EA%B3%A0%EC%98%81&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2020.01.01&de=2024.06.01&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20200101to20240601&is_sug_officeid=0&office_category=0&service_area=0"""
# Load the page
driver.get(search_url)
time.sleep(0.1)  # Wait for the page to load

# Scroll down until all articles are loaded
SCROLL_PAUSE_TIME = 0.1
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for the new page segment to load
    time.sleep(SCROLL_PAUSE_TIME)
    
    # Calculate the new scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # Break if the scroll height hasn't changed (no more content to load)
    if new_height == last_height:
        break
    last_height = new_height

# Use BeautifulSoup to extract article information
soup = BeautifulSoup(driver.page_source, 'html.parser')
all_articles = []
for item in tqdm(soup.select('.news_area'), desc='Extracting articles'):
    title_tag = item.select_one('.news_tit')
    title = title_tag.get_text(strip=True)
    link = title_tag['href']

    press_tag = item.select_one('.info_group > .press')
    press = press_tag.get_text(strip=True) if press_tag else ''

    date_tag = item.select_one('.info_group > span')
    date = date_tag.get_text(strip=True) if date_tag else ''

    all_articles.append({
        'title': title,
        'press': press,
        'date': date,
        'link': link
    })

# Close the browser
driver.quit()

# Function to extract article content
def get_article_content(url):
    try:
        article = Article(url, language='ko')
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print(f"Error fetching article: {e}")
        return None

# Function to extract keywords from content
def get_keywords(content):
    nouns = okt.nouns(content)
    filtered_nouns = [noun for noun in nouns if len(noun) > 1]
    return ', '.join(filtered_nouns)

# Fetch content and keywords for each article
for article in tqdm(all_articles, desc='Fetching article contents'):
    content = get_article_content(article['link'])
    article['content'] = content
    
    if content:
        keywords = get_keywords(content)
        article['keywords'] = keywords
    else:
        article['keywords'] = None

# Create a DataFrame
df = pd.DataFrame(all_articles)

# Function to parse dates and remove rows with invalid dates
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str).date()
    except ValueError:
        return None

# Apply date parsing and remove rows with invalid dates
df['date'] = df['date'].apply(parse_date)
df = df.dropna(subset=['date'])

# Save the DataFrame to an Excel file
excel_file = 'kohyoung1.xlsx'
df.to_excel(excel_file, index=False, engine='openpyxl')

print(f'Saved article information to {excel_file}')


[nltk_data] Downloading package stopwords to /home/jh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Fetching article contents:   3%|▎         | 101/4000 [01:11<3:38:54,  3.37s/it]

Error fetching article: Article `download()` failed with HTTPSConnectionPool(host='www.hankyung.com', port=443): Read timed out. on URL https://www.hankyung.com/finance/article/202001133536L


Fetching article contents:   3%|▎         | 105/4000 [01:13<1:10:35,  1.09s/it]

Error fetching article: Article `download()` failed with Response ended prematurely on URL http://yna.kr/AKR20200113121500527?did=1195m


Fetching article contents:   3%|▎         | 114/4000 [01:30<4:46:00,  4.42s/it]

...

Error fetching article: Article `download()` failed with HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/604240 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)'))) on URL http://www.sentv.co.kr/news/view/604240
Error fetching article: Article `download()` failed with HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/604697 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)'))) on URL http://www.sentv.co.kr/news/view/604697
Error fetching article: Article `download()` failed with HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/604695 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certifica

Fetching article contents:  93%|█████████▎| 3706/4000 [47:32<02:28,  1.98it/s]

Error fetching article: Article `download()` failed with HTTPSConnectionPool(host='www.sentv.co.kr', port=443): Max retries exceeded with url: /news/view/607294 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)'))) on URL http://www.sentv.co.kr/news/view/607294


Fetching article contents: 100%|██████████| 4000/4000 [50:08<00:00,  1.33it/s]


Saved article information to kohyoung1.xlsx


# date 타입 바꾸고 nat 값 제거 

In [ ]:
import pandas as pd

# 엑셀 파일 읽기
df = pd.read_excel("kohyoung.xlsx")

# 'date' 열을 datetime으로 변환, 유효하지 않은 날짜는 NaT로 처리
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# NaT 값을 가진 행 출력 (선택사항)
invalid_dates = df[df['date'].isna()]
if not invalid_dates.empty:
    print("유효하지 않은 날짜를 가진 행:")
    print(invalid_dates)

# NaT 값을 가진 행 제거
df_cleaned = df.dropna(subset=['date'])

# date 열에서 날짜만 추출
df_cleaned['date'] = df_cleaned['date'].dt.date

# 날짜순으로 정렬 (선택사항)
df_cleaned = df_cleaned.sort_values('date')

# 결과 출력
print(df_cleaned)

# 엑셀 파일로 저장
df_cleaned.to_excel('kohyoung2.xlsx', index=False)

print(f"처리된 데이터를 'kohyoung2.xlsx' 파일로 저장했습니다.")
print(f"총 {len(df_cleaned)} 개의 유효한 행이 저장되었습니다.")
print(f"첫 번째 날짜: {df_cleaned['date'].iloc[0]}, 마지막 날짜: {df_cleaned['date'].iloc[-1]}")

유효하지 않은 날짜를 가진 행:
                                       title       press date  \
19                  [고영의 문헌 속 ‘밥상’]고사리 꺾은 뜻은        경향신문  NaT   
134             [이주의 관.종]고영, HBM 검사장비로 영토 확장       아시아경제  NaT   
169   도쿄일렉트론·고영테크…'산단 지정 1년] 용인에 400개 기업 몰렸다        서울경제  NaT   
281                 고영"명품만 생존…K의료장비도 최고 될 것"  한국경제언론사 선정  NaT   
524                  반도체벨트 효과…소부장 기업들 용인에 둥지       아시아경제  NaT   
...                                      ...         ...  ...   
3728         조중생 前 교수·고광일 대표 ‘자랑스러운 서울인상’ 선정        문화일보  NaT   
3731                          [부음] 이동칠씨 별세 외        매일경제  NaT   
3732                           [부음] 송한경 별세 외        조선일보  NaT   
3736         조중생 前 교수·고광일 대표 ‘자랑스러운 서울인상’ 선정        문화일보  NaT   
3741         조중생 前 교수·고광일 대표 ‘자랑스러운 서울인상’ 선정        문화일보  NaT   

                                                   link  \
19    https://www.khan.co.kr/opinion/column/article/...   
134   https://view.asiae.co.kr/article/2023121914161...   
169         https://www.

/tmp/ipykernel_19161/500627688.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['date'] = df_cleaned['date'].dt.date


처리된 데이터를 'kohyoung2.xlsx' 파일로 저장했습니다.
총 3846 개의 유효한 행이 저장되었습니다.
첫 번째 날짜: 2022-03-06, 마지막 날짜: 2024-06-01


# unwanted data 처리

In [ ]:
import pandas as pd
import re

# Load the Excel file into a DataFrame
excel_file = 'kohyoung3.xlsx'
df = pd.read_excel(excel_file, engine='openpyxl')

# Define a function to check if a row contains any unwanted content
def has_unwanted_content(text):
    unwanted_patterns = [
        r'저작권', r'재배포', r'출처',r'사진',r'카카오스토리',r'시인',r'독서',r'희망이',r'소개',r'네이버',r'로그인',r'고양이', r'식단',# Korean keywords
        r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b',  # Email pattern
        r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'  # URL pattern
    ]
    return any(re.search(pattern, str(text)) for pattern in unwanted_patterns)

# Filter out rows with unwanted content
filtered_meere_df = df[~df['content'].apply(has_unwanted_content)]

# Drop rows containing NaN values
filtered_meere_df = filtered_meere_df.dropna()


# Reset index
filtered_meere_df.reset_index(drop=True, inplace=True)

# Save the filtered DataFrame to a new csv file
filtered_csv_file = 'kohyoung4.xlsx'
filtered_meere_df.to_excel(filtered_csv_file, index=False)

print(f'Filtered articles saved to {filtered_csv_file}.')

Filtered articles saved to kohyoung4.xlsx.


In [ ]:
import ast
df = pd.read_excel('kohyoung3.xlsx')
df.dropna(subset=['title', 'date', 'content','keyword'], inplace=True)
df = df[~df['content'].apply(lambda x: x =='카카오스토리(으)로 기사보내기 URL복사(으)로 기사보내기 이메일(으)로 기사보내기 다른 공유 찾기 기사스크랩하기' or
                             x =="가장 많이 읽힌 뉴스를 제공합니다. 집계 기준에 따라 최대 3일 전 기사까지 제공될 수 있습니다." or
                             x == "카카오톡(으)로 기사보내기 네이버밴드(으)로 기사보내기 네이버블로그(으)로 기사보내기 네이버라인(으)로 기사보내기 핀터레스트(으)로 기사보내기 URL복사(으)로 기사보내기" or
                             x == "라이브리 댓글 작성을 위해 JavaScript를 활성화해주세요" or
                             x == "이 기사를 공유합니다")]
df = df[~df['press'].apply(lambda x:x =="대구신문" or 
                           x == '뉴시스' )]
df = df[~df['keyword'].apply(lambda x:x == "네이버" or x == "카카오스토리" or x== '시인' or x=='영화' or x=='보험')]

df.to_excel('kohyoung4.xlsx', index=False, engine='openpyxl')


# pipline 연결 하고 데이터 정리

In [ ]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
# Continue with your other imports and script logic

import pandas as pd
from transformers import pipeline
import torch
df = pd.read_excel("kohyoung4.xlsx")


# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='snunlp/KR-FinBert-SC', device=device)
df['combined_text'] = df['title'] + ' ' + df['content']
def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    return result[0]['label']

# 감성 분석 및 새로운 열 생성
df['senti_ctext'] = df['combined_text'].apply(analyze_sentiment)
df['senti_keyword'] = df['keyword'].apply(analyze_sentiment)

# senti_value 칼럼 생성, sentiment_from_ctext와 sentiment_from_keyword이 같으면, 0, 다르면 1
df['match_status'] = df.apply(
    lambda row: 0 if row['senti_ctext'] == row['senti_keyword'] else 1, axis=1)
df['date'] = pd.to_datetime(df['date']).dt.date

# 파일 저장시, 칼럼값 지정

df[['date','title', 'content','link' ,'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_excel("kohyoung5.xlsx", index=False, engine='xlsxwriter')
print("Excel file created Successfully")
print(df.head(1))

2024-07-05 17:28:49.609693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 17:28:49.715677: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 17:28:49.716307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-05 17:28:49.891701: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-05 17:28:54.682558: W tensorflow/compiler/tf

Excel file created Successfully
                               title   press        date  \
0  넥슨지티, 코스닥시장 시가총액 순위 한주간 17계단 '점프'  핀포인트뉴스  2022-03-06   

                                                link  \
0  http://www.pinpointnews.co.kr/news/articleView...   

                                             content  \
0  코스닥 종목 중 넥슨지티 종목이 한주간 시총 순위가 가장 크게 뛰어 올랐다.\n\n...   

                                      keyword  \
0  종목, 계단, 코스닥, 순위, 위로, 바이오, 넥슨, 주간, 한국, 시가총액   

                                       combined_text senti_ctext  \
0  넥슨지티, 코스닥시장 시가총액 순위 한주간 17계단 '점프' 코스닥 종목 중 넥슨지...    positive   

  senti_keyword  match_status  
0       neutral             1  


# 데이터베이스에 저장

In [ ]:
import mysql.connector

conn = mysql.connector.connect(
    host = "database-1.cvqcay0g4c1o.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "jook",
    password = "************",
    database = "antking"
)
cursor = conn.cursor(buffered=True)

df = pd.read_excel('kohyoung5.xlsx')

insert_data_to_article_and_sentiment(conn, cursor, 'NAVER', '고영', df)

conn.close()

Inserted row 1 into article: 넥슨지티, 코스닥시장 시가총액 순위 한주간 17계단 '점프' (ID: 21298)
Inserted sentiment data for article ID 21298
Inserted keyword data for article ID 21298
Inserted row 2 into article: 넥슨지티, 코스닥시장 시가총액 순위 한주간 17계단 '점프' (ID: 21299)
Inserted sentiment data for article ID 21299
Inserted keyword data for article ID 21299
Inserted row 3 into article: 넥슨지티, 코스닥시장 시가총액 순위 한주간 17계단 '점프' (ID: 21300)
Inserted sentiment data for article ID 21300
Inserted keyword data for article ID 21300
Inserted row 4 into article: 넥슨지티, 코스닥시장 시가총액 순위 한주간 17계단 '점프' (ID: 21301)
Inserted sentiment data for article ID 21301
Inserted keyword data for article ID 21301
Inserted row 5 into article: [코스닥 마감]기관·외국인 쌍끌이 매도…880선 턱걸이 (ID: 21302)
Inserted sentiment data for article ID 21302
Inserted keyword data for article ID 21302
Inserted row 6 into article: 한국비엔씨 6.19% 급등 ...코스닥150 지수 구성종목 중 주가상승률 '선두' (ID: 21303)
Inserted sentiment data for article ID 21303
Inserted keyword data for article ID 21303
Inserted row 

In [ ]:
def insert_data_to_article_and_sentiment(conn, cursor, search_platform, search_keyword, df):
    try:
        # Article 테이블에 데이터 삽입
        for i, row in df.iterrows():
            sql_article = "INSERT INTO article (search_platform, search_keyword, pub_date, title, content, link) VALUES (%s, %s, %s, %s, %s, %s)"
            values_article = (search_platform, search_keyword, row['date'], row['title'], row['content'], row['link'])
            cursor.execute(sql_article, values_article)
            article_id = cursor.lastrowid  # 마지막으로 삽입된 article의 ID를 가져옴
            print(f"Inserted row {i+1} into article: {row['title']} (ID: {article_id})")
            # Sentiment 테이블에 데이터 삽입
            sql_sentiment = "INSERT INTO sentiment (article_id, sentiment_from_ctext, sentiment_from_keyword, match_status) VALUES (%s, %s, %s, %s)"
            values_sentiment = (article_id, row['senti_ctext'], row['senti_keyword'], row['match_status'])
            cursor.execute(sql_sentiment, values_sentiment)
            print(f"Inserted sentiment data for article ID {article_id}")
            # keyword 테이블에 데이터 삽입
            sql_keyword = "INSERT INTO keyword (article_id, keyword_set) VALUES (%s, %s)"
            values_keyword = (article_id, row['keyword'])
            cursor.execute(sql_keyword, values_keyword)
            print(f"Inserted keyword data for article ID {article_id}")
            conn.commit()
    except mysql.connector.Error as error:
        print(f"Error inserting data: {error}")